In [1]:
import pandas as pd 
import numpy as np
from os import listdir
from itertools import combinations, product  
from scipy.signal import find_peaks 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler 
from tqdm import tqdm

%matplotlib inline

import config
from file_io import read
from normalise import normalise_spectrums
from peak_search import peak_find, match_peaks
from theoretical_binding_list import create_binding_list

In [2]:
data_read_path = "..//Data//"
compound_list_path = data_read_path + "Compound Constraints//"
spectra_path = data_read_path + "Deconvoluted Spectra//"
spectra_files = listdir(spectra_path)
compounds_files = listdir(compound_list_path)

bound_file_path = spectra_path+spectra_files[0]
unbound_file_path = spectra_path+spectra_files[1]
compounds_file_path = compound_list_path+compounds_files[0]

In [3]:
bound_df, unbound_df, compounds = read(bound_file_path, unbound_file_path, compounds_file_path)

#extract information about Ub and the other compounds
other_compounds_df = compounds[compounds.Name != config.protein] #extract all compounds but Ub
protein_df = compounds[compounds.Name == config.protein] #extract Ub information

#Perform peak detection
#scale spectrums between 0 and 1
unbound_df, bound_df = normalise_spectrums(unbound_df, bound_df)

#find peaks 
peaks = peak_find(bound_df)

In [ ]:
binding_df = create_binding_list(other_compounds_df, protein_df)

In [19]:
#filter using max value
max_binding_mass = np.max(binding_df["Mass"]) #maximum possible mass from all theoretical binding sites
peaks = peaks[peaks["m/z"] < max_binding_mass] #remove incorrect peaks

,Formula,Mass,Level,Combs
0,C₃₇₈H₆₃₀N₁₀₅O₁₁₈S₁ + (NH₄)₅,8661.856,1,"(9,)"


In [ ]:
#match peaks 
binding_sites_df = match_peaks(peaks, binding_df)